In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io.wavfile as wav
import os
import pandas as pd
import matplotlib.pyplot as plt
import torchaudio
import glob
import math
import torch.nn as nn
import numpy as np
from bisect import bisect_left
import scipy as sp
import h5py


In [2]:
data_folder = "../datasets/generated_dataset/"
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
dropout = 0.1
batch_size = 128
fs = 16000

max_shift = 500 # in samples
guess_grid_size = max_shift


f = h5py.File("../datasets/generated_dataset/generated_dataset.hdf5","r")


X = f['input']
y = f['gt']

f_test = h5py.File("../datasets/generated_dataset/generated_dataset_valer.hdf5","r")
X_test = f_test['input']
y_test = f_test['gt']

#X = torch.load(os.path.join(data_folder,"input2.pt"))
#y = torch.load(os.path.join(data_folder,"gt2.pt"))
#X_test = torch.load(os.path.join(data_folder,"input.pt"))
#y_test = torch.load(os.path.join(data_folder,"gt.pt"))

class custom_dataset(Dataset):

    def __init__(self, X, y, idx_min=0,dataset_len=len(X)):
        self.X = X
        self.y = y
        self.dataset_len = dataset_len
        self.idx_min = idx_min

    def __getitem__(self, idx):
        return self.X[idx + self.idx_min],self.y[idx + self.idx_min]

    def __len__(self):
        return self.dataset_len

split_i = int(X.shape[0]*0.98)
dataset = custom_dataset(X,y, 0, split_i)
dataset_test = custom_dataset(X_test,y_test, 0 ,X_test.shape[0])
#dataset_test = custom_dataset(X,y, split_i, X.shape[0] - split_i)
#dataset, dataset_test = torch.utils.data.random_split(dataset, [0.98,0.02])
train_dl = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)


In [82]:
# to complex numbers
tt =next(iter(train_dl)) 
X = tt[0]
y = tt[1]

#def augment_shift(X,y):
X = torch.complex(X[:,0],X[:,1])
#print(.shape)
#augment 1, multiply each of the vectors with phase


max_shift = 100 # number of samples to max_shift
sample_length = 10000


imag_unit = torch.complex(torch.tensor(0.0),torch.tensor(1.0))
sample_shift = (torch.rand(X.shape[0],1,2)*2 - 1)*max_shift
phase_shift = (sample_shift/sample_length*2*torch.pi*imag_unit).exp()
new_abf = X*(-phase_shift*torch.arange(X.shape[1]).unsqueeze(1))
X = torch.stack([torch.real(new_abf),torch.imag(new_abf)], dim=1)
y - sample_shift[:,:,0] + sample_shift[:,:,1]
#return (X,y)






#torch.view_as_complex(abf.permute(1,2,0))
#abf.permute(1,2,0).stride(1)

#abf.shape



tensor([[ 3.5650e+01],
        [ 3.7420e+02],
        [-2.8142e+01],
        [ 2.4184e+01],
        [-1.8413e+02],
        [ 1.2527e+01],
        [-1.0289e+01],
        [ 1.4973e+02],
        [-9.2157e+01],
        [-9.6018e+01],
        [-6.5280e+01],
        [ 4.7821e+01],
        [ 7.3338e+01],
        [ 7.4025e+01],
        [-2.5373e+01],
        [ 1.7859e+02],
        [-2.2752e+02],
        [ 1.7075e+02],
        [-9.9035e+01],
        [ 1.3884e+02],
        [ 2.1542e+02],
        [-1.0822e+02],
        [ 8.5744e+01],
        [ 4.1299e+01],
        [-2.5045e+02],
        [ 2.0543e+02],
        [ 2.9728e+02],
        [ 1.1194e+02],
        [-1.8255e+02],
        [-1.0233e+01],
        [-3.3692e+01],
        [ 1.3247e+02],
        [-8.2365e+01],
        [ 7.4973e+01],
        [ 1.2076e+02],
        [ 1.0953e+02],
        [-4.5614e+01],
        [ 1.9838e+01],
        [ 1.4955e+02],
        [-7.9637e+01],
        [ 2.0058e+02],
        [ 3.2565e+02],
        [-1.7169e+02],
        [ 1